# Controlling With Noisy Measurements

In this exercise, we'll control the coaxial drone using measurements obtained by an altitude sensor. 
We will follow conventional notation and have $x_t$ represent the true state of the drone at time t, $z_t$ the measurement (of altitude) made at time t, and $\hat{x}_t$ the estimate of the true state at time t.

In [7]:
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

import numpy as np 
import math
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import jdc
from ipywidgets import interactive
from CoaxialDrone import CoaxialCopter
from PIDcontroller import PIDController_with_ff
from PathGeneration import flight_path
from DronewithPIDControllerParameters import DronewithPID

pylab.rcParams['figure.figsize'] = 10, 10

The sensor used by the quadrotor estimates the altitude with a weighted average defined as:

$$
\hat{x}_{t} = \alpha \hat{x}_{t-1} + (1-\alpha)z_t
$$

In [8]:
class Sensor:
    def __init__(self,
                 x_hat,            # State estimate
                 alpha             # Alpha value used in averaging
                ):
        '''
        Initialize the Sensor object with initial altitude estimation and the alpha value for the exponential averaging. 
        '''
        self.x_hat = x_hat         
        self.alpha = alpha         
        
    def measure(self, 
                x_t,               # Drone's true state
                sigma = 0.01       # Gaussian's variance
               ):
        '''
        We simulate a realistic altitude measurement by adding Gaussian noise to the true measurement.
        '''
        self.z_t = x_t + np.random.normal(0.0, sigma)
        return self.z_t


    def estimate(self, z_t):
        '''
        We then estimate the drone's altitude using the weighted average method.
        '''
        self.x_hat = self.alpha * self.x_hat + (1 - self.alpha) * z_t
        return self.x_hat

At $t=0$ our drone's position is $z=0$ and its velocity is $\dot{z}=0$. At this exact time, we ask it to change it's position by 1 meter in the negative direction. It's desired altitude will be $z=-1$. 

In [9]:
total_time = 10.0   # Total Flight time 
dt = 0.01           # A time interval between measurements 

t, z_path, z_dot_path, z_dot_dot_path =  flight_path(total_time, dt,'constant' )

Now let's compare three different estimates: 

- One that is actually impossible to obtain - that's the drone's true state.
- One where we use the sensor measurements as our estimated altitude.
- One where we use the recursive average of the sensor measurements as our estimated altitude.

We will make the comparison by looking at the path executed by the drone in each case.

First, let's see what the path looks like for the perfect state estimate, and compare that to an estimate which uses the measured values directly.

In [10]:
FlyingDrone = DronewithPID(z_path, z_dot_path, z_dot_dot_path, t, dt, Sensor)

In [11]:
interactive_plot = interactive(FlyingDrone.PID_controller_with_measured_values, 
                               k_p=(5.0, 35.0, 1),
                               k_d=(0.0, 10, 0.5), 
                               k_i=(0.0, 10, 0.5), 
                               mass_err =(0.7, 1.31, 0.01),
                               sigma=(0.0, 0.1, 0.001))
output = interactive_plot.children[-1]
output.layout.height = '800px'
interactive_plot

interactive(children=(FloatSlider(value=20.0, description='k_p', max=35.0, min=5.0, step=1.0), FloatSlider(value=5.0, description='k_d', max=10.0, step=0.5), FloatSlider(value=5.0, description='k_i', max=10.0, step=0.5), FloatSlider(value=1.0, description='mass_err', max=1.31, min=0.7, step=0.01), FloatSlider(value=0.05, description='sigma', max=0.1, step=0.001), Checkbox(value=False, description='use_measured_height'), Output(layout=Layout(height='800px'))), _dom_classes=('widget-interact',))

In this section, we will use the estimated value of the altitude based on the averaging to control the drone instead of relying only on the last measurement value.

In [12]:
interactive_plot = interactive(FlyingDrone.PID_controller_with_estimated_values, 
                               k_p=(5.0, 35.0, 1),
                               k_d=(0.0, 10, 0.5), 
                               k_i=(0.0, 10, 0.5), 
                               mass_err =(0.7, 1.31, 0.01),
                               sigma = (0.0, 0.1, 0.001),
                               alpha = (0.51, 0.99, 0.01))
output = interactive_plot.children[-1]
output.layout.height = '800px'
interactive_plot

interactive(children=(FloatSlider(value=20.0, description='k_p', max=35.0, min=5.0, step=1.0), FloatSlider(value=5.0, description='k_d', max=10.0, step=0.5), FloatSlider(value=5.0, description='k_i', max=10.0, step=0.5), FloatSlider(value=1.0, description='mass_err', max=1.31, min=0.7, step=0.01), FloatSlider(value=0.05, description='sigma', max=0.1, step=0.001), FloatSlider(value=0.75, description='alpha', max=0.99, min=0.51, step=0.01), Checkbox(value=False, description='use_estimated_height'), Output(layout=Layout(height='800px'))), _dom_classes=('widget-interact',))

# Questions:
---
* Is the magnitude of the steady-state error higher or lower when using the weighted average compared to the using the measured values directly?
* Does the drone converge at the desired altitude quicker or slower when using the weighted averaging for altitude estimation compared to using the direct altitude measurement?
---